# Run XGBoost

In [35]:
# importing all necessary libraries

# pip install -r requirements.txt

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller 
from statsmodels.tsa.statespace.sarimax import SARIMAX 
from datetime import datetime, timedelta
import statsmodels.api as sm
import gc
from pylab import rcParams
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import gc
import lightgbm as lgb
import numpy as np
import pandas as pd
import matplotlib.dates as mdates
import lightgbm as lgb
import calendar

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBRegressor
import multiprocessing as mp
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

# Load data

In [36]:
# Reading datasets

train_model = pd.read_csv('input/train_ready.csv')
test_model = pd.read_csv('input/test_ready.csv')

# Run model

In [37]:
# Train an XGBoost model for each product number

# Set Grid

param_grid = {
    'n_estimators': [300, 500, 700],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001]
}

train_data = train_model.loc[train_model['id'] < '2023']
test_data = train_model.loc[train_model['id'] >= '2023']

# Train an XGBoost model
    
xgb_model = XGBRegressor()

grid_search = GridSearchCV(estimator = xgb_model, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error')
    
grid_search.fit(train_data.drop(['id', 'inventory_units'], axis=1), train_data['inventory_units'])

best_xgb_model = grid_search.best_estimator_

In [38]:
# Evaluate the models and store results for the test period
    
predictions = best_xgb_model.predict(test_data.drop(['id',  'inventory_units'], axis=1))
    
test_data['predictions'] = predictions

# Write output

In [40]:
# Aggregate output at id level

test_data_output = test_data.groupby(['id']).sum().reset_index()

test_data_output

,id,inventory_units,sales_units,inventory_units_l4,inventory_units_l12,inventory_units_l52,product_number_100674,product_number_101661,product_number_102648,product_number_103635,...,week_number_5,week_number_50,week_number_51,week_number_52,week_number_53,week_number_6,week_number_7,week_number_8,week_number_9,predictions
0,202301-100674,2308.0,996.0,2198.0,1952.0,1915.0,18,0,0,0,...,1,0,0,0,0,1,1,1,1,1609.695923
1,202301-101661,93.0,57.0,183.0,90.0,63.0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,144.408875
2,202301-102648,155.0,21.0,90.0,55.0,160.0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,69.114395
3,202301-105609,24.0,40.0,63.0,147.0,71.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,105.549385
4,202301-107583,278.0,133.0,128.0,178.0,130.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,232.056671
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4081,202318-96726,171.0,187.0,155.0,107.0,148.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,145.711655
4082,202318-97713,55.0,2.0,51.0,108.0,181.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,62.586182
4083,202318-9870,101.0,96.0,144.0,121.0,64.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,141.478638
4084,202318-98700,142.0,63.0,91.0,194.0,109.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,119.848640


In [41]:
# Output

test_data_output.to_csv('output/test_results.csv', index = False)

# Done